In [2]:
import byzerllm
from byzerllm import ByzerLLM

# Initialize ByzerLLM
llm = ByzerLLM.from_default_model("deepseek_chat")

2024-08-29 08:16:46.534 | INFO     | byzerllm.utils.connect_ray:connect_cluster:48 - JDK 21 will be used (/Users/allwefantasy/.auto-coder/jdk-21.0.2.jdk/Contents/Home)...
2024-08-29 08:16:46,577	INFO worker.py:1564 -- Connecting to existing Ray cluster at address: 127.0.0.1:6379...
2024-08-29 08:16:46,598	INFO worker.py:1740 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 


In [3]:
from typing import List, Dict
from autocoder.common import AutoCoderArgs, SourceCode
from byzerllm.utils.client.code_utils import extract_code
import json
from loguru import logger

@byzerllm.prompt()
def extract_relevance_range_from_docs_with_conversation(
    conversations: List[Dict[str, str]], documents: List[str]
) -> str:
    """
    根据提供的文档和对话历史提取相关信息范围。

    输入:
    1. 文档内容:
    {% for doc in documents %}
    {{ doc }}
    {% endfor %}

    2. 对话历史:
    {% for msg in conversations %}
    <{{ msg.role }}>: {{ msg.content }}
    {% endfor %}

    任务:
    1. 分析最后一个用户问题及其上下文。
    2. 在文档中找出与问题相关的一个或多个重要信息段。
    3. 对每个相关信息段，确定其唯一的起始(start_str)和结束(end_str)字符串。

    输出要求:
    1. 返回一个JSON数组，每个元素包含"start_str"和"end_str"。
    2. start_str和end_str必须在原文中出现，但只能出现一次，并且两者之间不能重叠。
    3. 尽量选择简短但唯一的start_str和end_str。
    4. 如果没有相关信息，返回空数组[]。

    输出格式:
    严格的JSON数组，不包含其他文字或解释。

    示例:
    1.  文档：大象是陆地上最大的动物之一。它们生活在非洲和亚洲。
        问题：大象生活在哪里？
        返回：[{"start_str": "它们生活", "end_str": "在非洲和亚洲。"}]

    2.  文档：地球是太阳系第三行星，有海洋、沙漠，温度适宜，是已知唯一有生命的星球。
        问题：地球的特点是什么？
        返回：[{"start_str": "地球是太阳系", "end_str": "有生命的星球。"}]

    3.  文档：苹果富含维生素。香蕉含有大量钾元素。
        问题：橙子的特点是什么？
        返回：[]
    """  

# result = extract_relevance_range_from_docs_with_conversation.with_llm(llm).run(conversations=conversations, documents=documents)


# {"conversation":conversations, "doc":[doc.source_code]}
conversations = None
documents = None
with open("/tmp/rag.json", "r") as f:
    lines = f.read().split("\n")
    for i, line in enumerate(lines):
        if line:
            v = json.loads(line)
            conversations = v["conversation"]
            documents = v["doc"]

def process_range_doc(doc, max_retries=3):
    for attempt in range(max_retries):
        content = ""
        try:
            extracted_info = extract_relevance_range_from_docs_with_conversation.with_llm(
                llm
            ).run(
                conversations, [doc.source_code]
            )

            print(extracted_info)                                                             
            json_str = extract_code(extracted_info)[0][1]
            json_objs = json.loads(json_str)                                    
                                                
            for json_obj in json_objs:
                start_str = json_obj["start_str"]
                end_str = json_obj["end_str"]
                start_index = doc.source_code.index(start_str)
                end_index = doc.source_code.index(end_str) + len(end_str)
                content += doc.source_code[start_index:end_index] + "\n"  
                print(f"{start_str} - {end_str} : {doc.source_code[start_index:end_index]}")
            
            return SourceCode(
                module_name=doc.module_name, source_code=content.strip()
            )
        except Exception as e:
            if attempt < max_retries - 1:
                logger.warning(f"Error processing doc {doc.module_name}, retrying... (Attempt {attempt + 1}) attempts: {str(e)}")                                        
            else:
                logger.error(f"Failed to process doc {doc.module_name} after {max_retries} attempts: {str(e)}")
                return SourceCode(
                module_name=doc.module_name, source_code=content.strip()
            )

print(documents[0])
m = process_range_doc(SourceCode(
                module_name="test", source_code=documents[0]
            ))


print(m.source_code)


#/Users/allwefantasy/Downloads/docs/新版GMP解读.pptx#256
文本:新版GMP解读与分析
文本:安全监管注册处NO.1

#/Users/allwefantasy/Downloads/docs/新版GMP解读.pptx#258
文本:摘要
文本:基本情况与98版条款对比应对策略

#/Users/allwefantasy/Downloads/docs/新版GMP解读.pptx#257
文本:新版GMP基本情况
文本:历经5年修订两次公开征求意见2011年2月12日发布2011年3月1日起施行共有14 章、313 条

#/Users/allwefantasy/Downloads/docs/新版GMP解读.pptx#529
文本:新版GMP框架

#/Users/allwefantasy/Downloads/docs/新版GMP解读.pptx#494
文本:药品GMP的修订
文本:药品GMP基本要求五个附录：无菌药品、生物制品、血液制品、中药制剂、原料药98版药品GMP的另外三个附录暂不修订   中药饮片、放射性药品、医用气体修订重点：药品GMP基本要求、无菌药品附录   基本要求、无菌药品、生物制品和血液制品附录主要参考了欧盟与WHO的相关药品GMP要求，原料药附录主要参考了ICHQ7。

#/Users/allwefantasy/Downloads/docs/新版GMP解读.pptx#495
文本:GMP基本要求
文本:包括总则、质量管理、机构与人员、厂房与设施、设备、物料与产品、确认与验证、文件管理、生产管理、质量控制与质量保证、委托生产与委托检验、产品发运和召回、自检、附则等14章、54小节、313条，共计约3.2万字。

#/Users/allwefantasy/Downloads/docs/新版GMP解读.pptx#496
表单:
"98版GMP及相应的条款数量","","新修订药品GMP及相应条款数量",""
"1、总则","2条","1、总则","4条"
"","","2、质量管理","4节11条"
"2、机构与人员","5条","3、机构与人员","4节22条"
"3、厂房与设施","23条","4、厂房与设施","5节33条"
"4、设备","7条","5、设备","6节31条"
"5、物料","10条"

2024-08-29 08:17:02.284 | WARNING  | __main__:process_range_doc:94 - Error processing doc test, retrying... (Attempt 1) attempts: substring not found


[
    {
        "start_str": "质量系统",
        "end_str": "质量评价、风险管理的能力；"
    },
    {
        "start_str": "质量受权人",
        "end_str": "质量受权人制度。"
    },
    {
        "start_str": "质量管理",
        "end_str": "风险管理四部分"
    },
    {
        "start_str": "质量保证",
        "end_str": "质量保证的参与力度"
    },
    {
        "start_str": "质量控制",
        "end_str": "质量保证的参与力度"
    },
    {
        "start_str": "质量风险管理",
        "end_str": "风险管理四部分"
    }
]


2024-08-29 08:20:37.870 | WARNING  | __main__:process_range_doc:94 - Error processing doc test, retrying... (Attempt 2) attempts: Unterminated string starting at: line 487 column 9 (char 9333)


[
    {
        "start_str": "质量系统",
        "end_str": "质量评价、风险管理的能力；"
    },
    {
        "start_str": "质量体系",
        "end_str": "质量评价、风险管理的能力；"
    },
    {
        "start_str": "质量体系",
        "end_str": "质量评价、风险管理的能力；"
    },
    {
        "start_str": "质量体系",
        "end_str": "质量评价、风险管理的能力；"
    },
    {
        "start_str": "质量体系",
        "end_str": "质量评价、风险管理的能力；"
    },
    {
        "start_str": "质量体系",
        "end_str": "质量评价、风险管理的能力；"
    },
    {
        "start_str": "质量体系",
        "end_str": "质量评价、风险管理的能力；"
    },
    {
        "start_str": "质量体系",
        "end_str": "质量评价、风险管理的能力；"
    },
    {
        "start_str": "质量体系",
        "end_str": "质量评价、风险管理的能力；"
    },
    {
        "start_str": "质量体系",
        "end_str": "质量评价、风险管理的能力；"
    },
    {
        "start_str": "质量体系",
        "end_str": "质量评价、风险管理的能力；"
    },
    {
        "start_str": "质量体系",
        "end_str": "质量评价、风险管理的能力；"
    },
    {
        "start_str": "质量体系",
        "end_str": "质量评价、风险管理的能力；"
    

2024-08-29 08:20:44.402 | ERROR    | __main__:process_range_doc:96 - Failed to process doc test after 3 attempts: substring not found


[
    {
        "start_str": "质量系统",
        "end_str": "质量评价、风险管理的能力；"
    },
    {
        "start_str": "质量系统",
        "end_str": "质量评价、风险管理的能力；"
    },
    {
        "start_str": "质量系统",
        "end_str": "质量评价、风险管理的能力；"
    }
]

